# MLops project - Lung segmentation - 02 deployment

## Imort packages

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
 
import azureml.core

# display the core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.34.0


In [30]:
%%writefile score.py
import json
import numpy as np
import os
import tensorflow as tf
import cv2

from azureml.core.model import Model
import onnxruntime

def init():
    global model_path
    global output_names
    model_path = Model.get_model_path(model_name='lungs_autoencoder')

def run(image):
    data = np.array(json.loads(image)['image']).astype('float32')
    print(f'data: { data }')
    session = onnxruntime.InferenceSession(model_path)
    # first_input_name = session.get_inputs()[0].name
    # first_output_name = session.get_outputs()[0].name

    result = session.run(None, {"input": data})
    print(f'result {result}     {type(result)}')
    test = json.dumps({"image": result})
    test = bytes(test, encoding='utf8')
    return test


Overwriting score.py


### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container. While it depends on your model, the default of 1 core and 1 gigabyte of RAM is usually sufficient for many models. If you feel you need more later, you would have to recreate the image and redeploy the service.

In [31]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "lung images",  "method" : "autoencoder"}, 
                                               description='lung segmentation autoencoder')

### Deploy in ACI
Estimated time to complete: **about 2-5 minutes**

Configure the image and deploy. The following code goes through these steps:

1. Create environment object containing dependencies needed by the model using the environment file (`myenv.yml`)
1. Create inference configuration necessary to deploy the model as a web service using:
   * The scoring file (`score.py`)
   * envrionment object created in previous step
1. Deploy the model to the ACI container.
1. Get the web service HTTP endpoint.

In [32]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'lungs_autoencoder')


myenv = Environment.get(workspace=ws, name="lungs-env", version="1")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service_name = 'autoencoder-svc-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-24 13:48:17+00:00 Creating Container Registry if not exists.
2021-11-24 13:48:17+00:00 Registering the environment.
2021-11-24 13:48:18+00:00 Use the existing image.
2021-11-24 13:48:19+00:00 Submitting deployment to compute.
2021-11-24 13:48:24+00:00 Checking the status of deployment autoencoder-svc-5821..
2021-11-24 13:51:59+00:00 Checking the status of inference endpoint autoencoder-svc-5821.
Succeeded
ACI service creation operation finished, operation "Succeeded"
CPU times: user 806 ms, sys: 101 ms, total: 907 ms
Wall time: 3min 50s


In [33]:
print(service.scoring_uri)

http://bc733ac9-c304-42e5-aa47-95439f8a969c.westeurope.azurecontainer.io/score


## Test the model

### Download test data
Download the test data to the **./data/** directory

In [13]:
# azureml-core of version 1.0.72 or higher is required
from azureml.core import Workspace, Dataset

dataset_folder = os.path.join(os.getcwd(), "dataset")
os.makedirs(dataset_folder, exist_ok=True)

subscription_id = '31701647-d61e-4ee8-a1ae-e4fc5d71e6f6'
resource_group = 'MLOps-Home-Grp_8'
workspace_name = 'mlopslungs'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='lung_data')
dataset.download(target_path=dataset_folder, overwrite=True)

['/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopslungs/code/Users/brecht.willems2/dataset/Lung_images/CHNCXR_0001_0.png',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopslungs/code/Users/brecht.willems2/dataset/Lung_images/CHNCXR_0002_0.png',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopslungs/code/Users/brecht.willems2/dataset/Lung_images/CHNCXR_0003_0.png',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopslungs/code/Users/brecht.willems2/dataset/Lung_images/CHNCXR_0004_0.png',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopslungs/code/Users/brecht.willems2/dataset/Lung_images/CHNCXR_0005_0.png',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopslungs/code/Users/brecht.willems2/dataset/Lung_images/CHNCXR_0006_0.png',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopslungs/code/Users/brecht.willems2/dataset/Lung_images/CHNCXR_0007_0.png',
 '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlopslungs/code/Users/brecht.willems2/dataset/Lung_ima

## Predict test data
Feed the test dataset to the model to get predictions.

The following code goes through these steps: 1. Send the data as a JSON array to the web service hosted in ACI.

Use the SDK's run API to invoke the service. You can also make raw calls using any HTTP tool such as curl.

In [34]:
import glob 
import cv2
import json

# reading image
path = glob.glob("./dataset/Lung_images/CHNCXR_0001_0.png") 
img = cv2.imread(path[0]) 
im_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
resize = cv2.resize(im_rgb, (400,400))

# normalize image
image = np.array(resize)
image = image.reshape(-1, 400, 400, 1)
image = image.astype('float32') / 255

# json dump
test = json.dumps({"image": image.tolist()})
test = bytes(test, encoding='utf8')

y_hat = service.run(test)
print(image.shape)

Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '49', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 24 Nov 2021 13:52:14 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '44c6cd42-9dc1-46ce-b007-78b7d0844908', 'X-Ms-Run-Function-Failed': 'True'}
Content: b"Object of type 'ndarray' is not JSON serializable"



WebserviceException: WebserviceException:
	Message: Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.
Response Code: 502
Headers: {'Connection': 'keep-alive', 'Content-Length': '49', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 24 Nov 2021 13:52:14 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '44c6cd42-9dc1-46ce-b007-78b7d0844908', 'X-Ms-Run-Function-Failed': 'True'}
Content: b"Object of type 'ndarray' is not JSON serializable"
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Received bad response from service. More information can be found by calling `.get_logs()` on the webservice object.\nResponse Code: 502\nHeaders: {'Connection': 'keep-alive', 'Content-Length': '49', 'Content-Type': 'text/html; charset=utf-8', 'Date': 'Wed, 24 Nov 2021 13:52:14 GMT', 'Server': 'nginx/1.10.3 (Ubuntu)', 'X-Ms-Request-Id': '44c6cd42-9dc1-46ce-b007-78b7d0844908', 'X-Ms-Run-Function-F

In [35]:
service.get_logs()

'[0.6274510025978088], [0.6745098233222961], [0.729411780834198], [0.729411780834198], [0.729411780834198], [0.7137255072593689], [0.6901960968971252], [0.6784313917160034], [0.6627451181411743], [0.615686297416687], [0.5647059082984924], [0.5372549295425415], [0.5215686559677124], [0.5098039507865906], [0.501960813999176], [0.4901960790157318], [0.4901960790157318], [0.48627451062202454], [0.48627451062202454], [0.47843137383461], [0.47843137383461], [0.47058823704719543], [0.47058823704719543], [0.46666666865348816], [0.4627451002597809], [0.4627451002597809], [0.45490196347236633], [0.4431372582912445], [0.4274509847164154], [0.4156862795352936], [0.4117647111415863], [0.40392157435417175], [0.3960784375667572], [0.38823530077934265], [0.37254902720451355], [0.3450980484485626], [0.3333333432674408], [0.3137255012989044]], [[0.11372549086809158], [0.05882352963089943], [0.03921568766236305], [0.0313725508749485], [0.027450980618596077], [0.0235294122248888], [0.01568627543747425], [